# AnthropicToolCallState

## Example: Create TOOL_USE state using AnthropicToolUseState

This is actually 2 actions within the same state. The first action is to call the tool and the second action is to pass the result to the LLM and return a response.

```lua
stateDiagram-v2
    INIT --> TOOL_CALL
    TOOL_CALL --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> TOOL_CALL
    TOOL_CALL --> FINAL
```


In [1]:
from gai.asm import AsyncStateMachine
from gai.asm import Monologue
from gai.mcp.client import McpAggregatedClient

monologue = Monologue()
monologue.add_user_message("What time is it now?")
monologue.add_assistant_message(
    [
        {
            "citations": None,
            "text": "I'll get the current time for you.",
            "type": "text",
        },
        {
            "id": "toolu_012CyVmXRfMRzxoYBtmoeF1s",
            "input": {"format": "YYYY-MM-DD HH:mm:ss"},
            "name": "current_time",
            "type": "tool_use",
        },
    ]
)

with AsyncStateMachine.StateMachineBuilder(
    """
    INIT --> TOOL_USE
    TOOL_USE--> FINAL
    """
) as builder:
    fsm = builder.build(
        {
            "INIT": {
                "input_data": {
                    "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                    "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                }
            },
            "TOOL_USE": {
                "module_path": "gai.asm.states",
                "class_name": "AnthropicToolUseState",
                "title": "TOOL_USE",
                "input_data": {
                    "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                    "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                },
                "output_data": ["streamer", "get_assistant_message"],
            },
            "FINAL": {
                "output_data": ["monologue"],
            },
        },
        get_llm_config=lambda state: {
            "client_type": "anthropic",
            # "model": "claude-opus-4-20250514",
            "model": "claude-sonnet-4-20250514",
            "max_tokens": 32000,
            "temperature": 0.7,
            "top_p": 0.95,
            "tools": True,
        },
        get_mcp_client=lambda state: McpAggregatedClient(["mcp-time"]),
        monologue=monologue,
    )

## Step 2: INIT --> TOOL_CALL

await fsm.run_async()
async for chunk in fsm.state_bag["streamer"]:
    if (isinstance(chunk,str)):
        print(chunk, end='', flush=True)
print("\n\n")

## Step 4: Print the state history
for message in fsm.state_history[1]["output"]["monologue"].list_messages():
    print(
        f"{message.header.timestamp} {message.header.sender} > {message.body.content}"
    )

The current time is **June 16, 2025 at 10:48:26 AM UTC**.

If you'd like to know the time in a specific timezone, please let me know which timezone you're interested in!


1750070904.2777271 User > What time is it now?
1750070904.2779763 Assistant > [{'citations': None, 'text': "I'll get the current time for you.", 'type': 'text'}, {'id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'input': {'format': 'YYYY-MM-DD HH:mm:ss'}, 'name': 'current_time', 'type': 'tool_use'}]
1750070906.3662696 User > [{'type': 'tool_result', 'tool_use_id': 'toolu_012CyVmXRfMRzxoYBtmoeF1s', 'content': 'Current UTC time is 2025-06-16 10:48:26, and the time in UTC is 2025-06-16 10:48:26.'}]
1750070909.7755535 Assistant > [{'citations': None, 'text': "The current time is **June 16, 2025 at 10:48:26 AM UTC**.\n\nIf you'd like to know the time in a specific timezone, please let me know which timezone you're interested in!", 'type': 'text'}]
